use Crypto library to generate two big prime number (512 bits)

In [44]:
from Cryptodome.Util import number

n_length = 512

p = number.getPrime(n_length)
q = number.getPrime(n_length)

n = p * q
phi_n = (p - 1) * (q - 1)

find a number `e` that $1<e<\varphi(n)$ and $gcd( \varphi (n),e) = 1$ as a part of public key.

In [45]:
e = 2
while True:
    e = number.getRandomRange(2, phi_n)
    if number.GCD(e, phi_n) == 1:
        break
print(e)

34147124941354239932744469664518003846542069677905631193751465240454772051071551223331106258466732179065827454179678318123177701455257686425709898899598209017301080749672799557750388259306375669933579699176391168061294855625647701805698333772769243851923286855507493707103806201812013751130299649916031277607


find a number `d` that $ed \equiv gcd( \varphi (n),e) \mod \varphi (n)$ or $ed \equiv 1\mod \varphi (n)$.

use exgcd method to solve equation $ed + k \varphi(n) = 1$

d is a part of private key.


public key: (n, e)
private key: (n, d)

In [46]:
def exgcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = exgcd(b % a, a)
        return g, y - (b // a) * x, x

_, d, _ = exgcd(e, phi_n)
d = abs(d)

print(d)

22202740070900825661240784140061220079232257358528526214220583718563425172735961247636143401565617118255645914366475268905949841544050336623687543356140070748064113315096765697139813183687283783166857865065771256859659713333615389235555549662247574903320124481486599454751928621758730249540674948876714636183


convert str `M` to hex, then to int.

In [47]:
M = "Top secret?"

m_hex = M.encode().hex()
m = int(m_hex, 16)
print(m)

102076019900887746094658623


calculate $c = m^d \mod n$

In [48]:
def QuickPower(a, n, p): # 快速幂算法
    tmp = a
    ret = 1
    while n > 0:
        if n&1:
            ret = (ret * tmp) % p
        tmp = (tmp * tmp) % p
        n>>=1
    return ret

c = QuickPower(m, e, n)
print(c)

33445738306313427667969456734837185600130574784236451630427208058705235963885427077798805487167465413023955642468009407046518610617139586906631424376689612213080144462917845422780575355330871582914837535592857084975549239254271158631582886535586792919293673416170759038019334894401837145631120699332406060211


calculate $ m' = c^e \mod n$

In [50]:
m_ = QuickPower(c, d, n)

m_ = m_.to_bytes(n_length,byteorder='little').decode()
print(m_.strip("\0")[::-1])


Top secret?


sign a message, use private key to encrypt the message.

In [52]:
S = QuickPower(m, d, n)
print(S)

55152997352602290499360851433496056157501653270435849601561918070440077364490798120664785216313998667354217314351440167709774241395593293479490792365203656176030354413261454019652478667358725264016337552959624789367087772756093318067488639628946492394909958947267947941939397131526962127528222366532554944516


verify a signature, use corresponding public key to decrypt the signature and compare it with plain text.

In [53]:
if QuickPower(S, e, n) == m:
    print("valid")




valid


In [20]:

def QuickPower(a, n, p): # 快速幂算法
    tmp = a
    ret = 1
    while n > 0:
        if n&1:
            ret = (ret * tmp) % p
        tmp = (tmp * tmp) % p
        n>>=1
    return ret
n = 0xA75AC9D50C18210023D5970FEBAEDD5C686B6B8F5060137A81CB97EE8E8A61944B2679F604A72AFBA4DA56BBEEA0A4F07B8A7F551F4793610D6E71513A2524082F8CE1F789D692CFAFB3A73F30EDB5DF21AEFEF54417FDD863D92FD3815A6B5FD347B0ACF2AB3B24794F1FC72EEAB9153A7C184C69B3B52059095E29C363E62E465BAA9490490EB9F0F54AA1092F7C344DD0BC00C506557906CEA2D010F14843E8B95AB59555BD31D21B3D86BEA1EC0D12DB2C9924AD47C26F03E67A70B570CCCD272CA58C8EC2183C92C92E736F0610569340AAA3C552FBE5C505D669685C06B9EE5189E18A0E414D9B92900A89E9166BEFEF75BE7A46B8E3478A1D1C2EA74F
S = 0x257dcba114f605ba95cc782e2756da9830ebeb9707a351e0597b04ecdbc3b26fe03838f68c0aaa27a9ea685fedfaae33a03f1a4ada4b0938c0cbd7100b521e8411b611ae5d529b010f67901b2045514c0de2b8b2617f6c4fcbbb8877bd4388887163d9d8ed0337956d63fbe0e4d52ebd15083f097fccf019fcab1fe14ef0a705d577d24b7747cdd2043440020ec50bd91e0089197f192e3c4116e66b0b1dc20d6c0cf2390e6591fc7b74ff17e95fcf2c1ddc1b6ddccfc0082be24ebb09d0e6851aa73f14d33160a86ccfabc37577d7e168031f5fd81f32150ecf7ec3a97a11cd5d05cfd665e337e9e2d7d61a97bd1a6b1ecddff98740c9ff013ab34eb8ff5512
e = 65537

d = QuickPower(S, e, n)

print(hex(d))

0x1ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff003031300d060960864801650304020105000420b4a346a7e966ff17ff6d2a1e8673843e9bc424122a26f7600e652ba861c17928
